# Monthly Fracture Maps

Next, we will pair the stress maps with a monthly time series of fracture maps, courtesy of Trystan's paper

## Import Packages

In [1]:
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rxr

from shapely.geometry import Polygon
from shapely.geometry import Point
import matplotlib.pyplot as plt

import json
import blue_ice_tools as bit
import itslivetools

## Load in Shapefile, Velocity Data, and compute Stress

In [2]:
# Read in shapefile of Shirase Glacier
shirase_shape = gpd.read_file('../data/shirase-glacier/shirase.shp')

# Load in ITS_LIVE velocities with shape
shirase_dc = bit.get_data_cube(shape=shirase_shape, epsg=3031)

# Create a list of variable names
vars = ['eps_eff', 'eps_xx', 'eps_yy', 'sigma_vm', 'sigma1', 'sigma2']

# Call stress function, save outputs as new data vars
shirase_dc[vars] = bit.compute_strain_stress(shirase_dc.vx, shirase_dc.vy, rotate=True)
shirase_dc

<xarray.Dataset> Size: 750MB
Dimensions:   (mid_date: 97, y: 655, x: 328)
Coordinates:
  * x         (x) float64 3kB 1.344e+06 1.344e+06 ... 1.383e+06 1.384e+06
  * y         (y) float64 5kB 1.649e+06 1.65e+06 ... 1.728e+06 1.728e+06
    mapping   int64 8B 0
  * mid_date  (mid_date) datetime64[ns] 776B 2015-01-31 ... 2023-01-31
Data variables:
    v         (mid_date, y, x) float32 83MB dask.array<chunksize=(97, 655, 328), meta=np.ndarray>
    vx        (mid_date, y, x) float32 83MB dask.array<chunksize=(97, 655, 328), meta=np.ndarray>
    vy        (mid_date, y, x) float32 83MB dask.array<chunksize=(97, 655, 328), meta=np.ndarray>
    eps_eff   (mid_date, y, x) float32 83MB dask.array<chunksize=(97, 655, 328), meta=np.ndarray>
    eps_xx    (mid_date, y, x) float32 83MB dask.array<chunksize=(97, 655, 328), meta=np.ndarray>
    eps_yy    (mid_date, y, x) float32 83MB dask.array<chunksize=(97, 655, 328), meta=np.ndarray>
    sigma_vm  (mid_date, y, x) float32 83MB dask.array<chunksize=(97, 655, 328), meta=np.ndarray>
    sigma1    (mid_date, y, x) float32 83MB dask.array<chunksize=(97, 655, 328), meta=np.ndarray>
    sigma2    (mid_date, y, x) float32 83MB dask.array<chunksize=(97, 655, 328), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.8
    GDAL_AREA_OR_POINT:         Area
    author:                     ITS_LIVE, a NASA MEaSUREs project (its-live.j...
    autoRIFT_parameter_file:    http://its-live-data.s3.amazonaws.com/autorif...
    datacube_software_version:  1.0
    date_created:               26-Sep-2023 13:16:09
    ...                         ...
    s3:                         s3://its-live-data/datacubes/v2/S70E030/ITS_L...
    skipped_granules:           s3://its-live-data/datacubes/v2/S70E030/ITS_L...
    time_standard_img1:         UTC
    time_standard_img2:         UTC
    title:                      ITS_LIVE datacube of image pair velocities
    url:                        https://its-live-data.s3.amazonaws.com/datacu...

## Load in Fracture Maps

Thank you Trys for providing monthly crevasse maps that have a considerable overlap, from July 2018 - January 2023

In [30]:
fracture = xr.open_dataset('../data/shirase-glacier/shirase-fracture-clipped.nc')
fracture

<xarray.Dataset> Size: 93MB
Dimensions:        (mid_date: 54, y: 655, x: 328)
Coordinates:
  * mid_date       (mid_date) datetime64[ns] 432B 2018-07-31 ... 2023-01-31
    spatial_ref    int64 8B ...
  * x              (x) float64 3kB 1.344e+06 1.344e+06 ... 1.383e+06 1.384e+06
  * y              (y) float64 5kB 1.649e+06 1.65e+06 ... 1.728e+06 1.728e+06
    mapping        int64 8B ...
Data variables:
    fracture_conf  (mid_date, y, x) float64 93MB ...

## Time Series of Crevasse Field

For this example, we will create a GIF of a crevasse onset region from the area boxed in red below

![Crevasse Onset Region](../figures/shirase-glacier/onset-region-details.png)

In [36]:
x, y = (1.353e6, 1.675e6)
ds = shirase_dc.sel(x=slice(1.344e6,1.36e6), y=slice(1.67e6, 1.683e6)).compute()
figsize = (18,5)

In [37]:
# Add a buffer area around
buffer = 6e4
spot = ds.sel(x=slice(x-buffer, x+buffer), y=slice(y-buffer, y+buffer),).mean(['x','y'])

lw = .8
for i in range(len(ds.mid_date)):
    mid_date = ds.mid_date[i].values
    
    # Plot fracture map and stress side by side
    fig, axs = plt.subplots(ncols=3, figsize=figsize, layout='constrained')
    
    spot.sigma_vm.plot(ax=axs[0], x='mid_date', color='steelblue', lw=lw)
    axs[0].scatter(y=spot.sigma_vm[i], x=mid_date, color='steelblue')
    
    ax2 = axs[0].twinx()
    # spot.fracture_conf.plot(ax=ax2, x='mid_date', color='tomato', lw=lw)
    # ax2.scatter(y=spot.fracture_conf[i], x=mid_date, color='tomato')
    
    # Plot stress on same day
    b = ds.sigma_vm[i].plot(ax=axs[1], cmap='magma', vmax=1000)
    axs[1].set_title(None)
    axs[1].set_aspect('equal')
    b.colorbar.set_label('Tensile Stress ($kPa$)')
    axs[1].axvline(x=x, c='blue')
    axs[1].axhline(y=y, c='blue')
    
    # # Plot fracture 
    # a = ds.fracture_conf[i].plot(ax=axs[2], cmap='inferno')
    # axs[2].set_title(None)
    # axs[2].set_aspect('equal')
    # a.colorbar.set_label('Fracture Confidence [0:1]')
    # axs[2].axvline(x=x, c='blue', lw=1)
    # axs[2].axhline(y=y, c='blue', lw=1)
    
    plt.suptitle(f'Shirase Galcier Crevasse and Stresses on {np.datetime64(mid_date, "D")}')
    plt.close()
    # plt.savefig(f'../figures/tib-headwall-{i}.png')

In [31]:
## Next step here:
## Plot monthly map at a single point from the crevasse field, make a .gif
## Identical to the one for my slideshow

In [32]:
## After that, show the lagrangian frame func
## Make the failure map gif of the cool crevasse